In [1]:
import numpy as np
import pickle
import scipy
from scipy.stats import entropy
from sklearn.preprocessing import normalize
import json
import random
import time
from nltk.tokenize import sent_tokenize
from scipy.stats import ttest_ind, ttest_rel
import seaborn as sns
import pandas as pd
import csv
from collections import Counter
from sklearn.metrics import balanced_accuracy_score, accuracy_score

In [2]:
outputdir = '../output/abcd/'
corefdir = '../output/halueval_coref/'
knowgldir = '../dataset/wiki/'
oriDocDir = '../dataset/'
datadir = '../../HaluEval/data/'
abcddir = '../output/abcd/'
imagedir = '../images/'

In [65]:
#halueval summarization
with open(imagedir + "image_hs.pickle", "rb") as f:
    img = pickle.load(f)
with open(imagedir + "image_hs_self.pickle", "rb") as f:
    img_self = pickle.load(f)
with open(imagedir + "results_hs.pickle", "rb") as f:
    res = pickle.load(f)

In [3]:
#halueval dialogue
with open(imagedir + "image_hd.pickle", "rb") as f:
    img = pickle.load(f)
with open(imagedir + "image_hd_self.pickle", "rb") as f:
    img_self = pickle.load(f)
with open(imagedir + "results_hd.pickle", "rb") as f:
    res = pickle.load(f)

In [74]:
#halueval qa
with open(imagedir + "image_hq.pickle", "rb") as f:
    img = pickle.load(f)
with open(imagedir + "image_hq_self.pickle", "rb") as f:
    img_self = pickle.load(f)
with open(imagedir + "results_hq.pickle", "rb") as f:
    res = pickle.load(f)

In [27]:
#cgs
with open(imagedir + "image_cogensumm.pickle", "rb") as f:
    img = pickle.load(f)
with open(imagedir + "image_cogensumm_self.pickle", "rb") as f:
    img_self = pickle.load(f)
with open(imagedir + "results_cgs.pickle", "rb") as f:
    res = pickle.load(f)

In [48]:
#factcc
with open(imagedir + "image_factcc.pickle", "rb") as f:
    img = pickle.load(f)
with open(imagedir + "image_factcc_self.pickle", "rb") as f:
    img_self = pickle.load(f)
with open(imagedir + "results_factcc.pickle", "rb") as f:
    res = pickle.load(f)

In [34]:
#xsf
with open(imagedir + "image_xsumfaith.pickle", "rb") as f:
    img = pickle.load(f)
with open(imagedir + "image_xsumfaith_self.pickle", "rb") as f:
    img_self = pickle.load(f)
with open(imagedir + "results_xsf.pickle", "rb") as f:
    res = pickle.load(f)

In [41]:
#polytope
with open(imagedir + "image_polytope.pickle", "rb") as f:
    img = pickle.load(f)
with open(imagedir + "image_polytope_self.pickle", "rb") as f:
    img_self = pickle.load(f)
with open(imagedir + "results_polytope.pickle", "rb") as f:
    res = pickle.load(f)

In [57]:
#frank
with open(imagedir + "image_frank.pickle", "rb") as f:
    img = pickle.load(f)
with open(imagedir + "image_frank_self.pickle", "rb") as f:
    img_self = pickle.load(f)
with open(imagedir + "results_frank.pickle", "rb") as f:
    res = pickle.load(f)

In [4]:
gd_labels = res[0]["labels"]
ins = len(img)

In [5]:
print(len(gd_labels))
print(len(img_self))
print(ins)

2000
2000
2000


In [6]:
# label: e, c, n
e_map = []
e_selfmap = []
c_map = []
c_selfmap = []
for i in range(ins):
    e_map.append(img[i][i][0])
    c_map.append(img[i][i][1])

    e_selfmap.append(img_self[i][i][0])
    c_selfmap.append(img_self[i][i][1])

In [7]:
c_selfmap_avg = []
e_selfmap_avg = []
for i in range(ins):
    c_selfmap_avg.append(np.mean(c_selfmap[i],axis=1).reshape(-1,1))
    e_selfmap_avg.append(np.mean(e_selfmap[i],axis=1).reshape(-1,1))

In [8]:
c_mapdiff = []
e_mapdiff = []
for i in range(ins):
    c_mapdiff.append(np.array(c_map[i]) - c_selfmap_avg[i])
    e_mapdiff.append(np.array(e_map[i]) - e_selfmap_avg[i])

In [9]:
def getFinal(value_map, op1="max",op2="avg"):
    outputs = []
    for idx in range(ins):
        submap = value_map[idx]
        op1_map = []
        if op1 == "max":
            op1_map = np.max(submap,axis=0)
        elif op1 == "min":
            op1_map = np.min(submap,axis=0)
        elif op1 == "avg":
            op1_map = np.mean(submap,axis=0)
        if op2 == "max":
            outputs.append(np.max(op1_map))
        elif op2 == "min":
            outputs.append(np.min(op1_map))
        elif op2 == "avg":
            outputs.append(np.mean(op1_map))
    return outputs

In [12]:
def getP(op1, op2):
    c_pr = getFinal(c_map,op1,op2)
    c_cal_pr = getFinal(c_mapdiff,op1,op2)
    e_pr = getFinal(e_map,op1,op2)
    e_cal_pr = getFinal(e_mapdiff,op1,op2)

    test_size = ins
    best_th_c = 0
    best_c = 0
    
    best_th_c_cal = 0
    best_c_cal = 0
    
    best_th_e = 0
    best_e = 0
    
    best_th_e_cal = 0
    best_e_cal = 0
    
    for th_step in range(99):
        th = 0.01 + th_step*0.01
    
        pr_c = []
        pr_c_cal = []
        pr_e = []
        pr_e_cal = []
        
        for i in range(test_size):
            if c_pr[i] < th:
                pr_c.append(1)
            else:
                pr_c.append(0)
    
            if c_cal_pr[i] < th:
                pr_c_cal.append(1)
            else:
                pr_c_cal.append(0)
    
            if e_pr[i] > th:
                pr_e.append(1)
            else:
                pr_e.append(0)
    
            if e_cal_pr[i] > th:
                pr_e_cal.append(1)
            else:
                pr_e_cal.append(0)

    
        # c_acc = accuracy_score(gd_labels[:test_size],pr_c)*100
        # c_cal_acc = accuracy_score(gd_labels[:test_size],pr_c_cal)*100
        # e_acc = accuracy_score(gd_labels[:test_size],pr_e)*100
        # e_cal_acc = accuracy_score(gd_labels[:test_size],pr_e_cal)*100
        # x_acc = accuracy_score(gd_labels[:test_size],pr_x)*100
        # x_cal_acc = accuracy_score(gd_labels[:test_size],pr_x_cal)*100

        c_acc = balanced_accuracy_score(gd_labels[:test_size],pr_c)*100
        c_cal_acc = balanced_accuracy_score(gd_labels[:test_size],pr_c_cal)*100
        e_acc = balanced_accuracy_score(gd_labels[:test_size],pr_e)*100
        e_cal_acc = balanced_accuracy_score(gd_labels[:test_size],pr_e_cal)*100

    
        if c_acc > best_c:
            best_th_c = th
            best_c = c_acc
        if c_cal_acc > best_c_cal:
            best_th_c_cal = th
            best_c_cal = c_cal_acc
        if e_acc > best_e:
            best_th_e = th
            best_e = e_acc
        if e_cal_acc > best_e_cal:
            best_th_e_cal = th
            best_e_cal = e_cal_acc

    return best_e, best_e_cal

In [33]:
#CGS

ops = ["max","min","avg"]

best_e = 0
best_e_cal = 0

for o1 in ops:
    for o2 in ops:
        e, e_cal = getP(o1,o2)
        print(o1, o2, e_cal, e)
        if e > best_e:
            best_e = e
        if e_cal > best_e_cal:
            best_e_cal = e_cal

print("best e_cal:", best_e)
print("best e:", best_e_cal)

max max 57.02214452214452 56.701631701631705
max min 75.247668997669 71.43065268065267
max avg 76.51515151515152 74.92715617715618
min max 50.641025641025635 50.16025641025641
min min 50.0 50.0
min avg 50.16025641025641 50.0
avg max 51.6462703962704 51.42773892773893
avg min 69.14335664335664 50.80128205128205
avg avg 57.98368298368298 52.28729603729604
best e_cal: 76.51515151515152
best e: 74.92715617715618


In [40]:
#XSF

ops = ["max","min","avg"]

best_e = 0
best_e_cal = 0

for o1 in ops:
    for o2 in ops:
        e, e_cal = getP(o1,o2)
        print(o1, o2, e_cal, e)
        if e > best_e:
            best_e = e
        if e_cal > best_e_cal:
            best_e_cal = e_cal

print("best e_cal:", best_e)
print("best e:", best_e_cal)

max max 66.29464285714286 66.76167582417582
max min 66.13324175824175 66.46634615384616
max avg 66.33928571428571 66.76167582417582
min max 50.76923076923077 50.0
min min 50.76923076923077 50.0
min avg 50.76923076923077 50.0
avg max 62.108516483516475 52.541208791208796
avg min 61.99175824175824 52.245879120879124
avg avg 62.153159340659336 52.58585164835166
best e_cal: 66.33928571428571
best e: 66.76167582417582


In [47]:
#poly

ops = ["max","min","avg"]

best_e = 0
best_e_cal = 0

for o1 in ops:
    for o2 in ops:
        e, e_cal = getP(o1,o2)
        print(o1, o2, e_cal, e)
        if e > best_e:
            best_e = e
        if e_cal > best_e_cal:
            best_e_cal = e_cal

print("best e_cal:", best_e)
print("best e:", best_e_cal)

max max 56.134578209188504 52.22309052770123
max min 59.09801340846461 60.01933122198001
max avg 66.183 61.750915148274586
min max 50.12339077859581 50.0
min min 50.0 50.0
min avg 50.0 50.0
avg max 60.45942499897174 58.59005470324517
avg min 60.360712376095094 53.905318142557476
avg avg 61.48356846131699 57.454859540163696
best e_cal: 66.183
best e: 61.750915148274586


In [54]:
#factcc

ops = ["max","min","avg"]

best_e = 0
best_e_cal = 0

for o1 in ops:
    for o2 in ops:
        e, e_cal = getP(o1,o2)
        print(o1, o2, e_cal, e)
        if e > best_e:
            best_e = e
        if e_cal > best_e_cal:
            best_e_cal = e_cal

print("best e_cal:", best_e)
print("best e:", best_e_cal)

max max 89.29120035110817 86.17877258430254
max min 89.29120035110817 86.2921512691098
max avg 89.51795772072269 86.2921512691098
min max 50.68027210884354 50.0
min min 50.56689342403629 50.0
min avg 50.56689342403629 50.0
avg max 74.49711067222589 52.80886548167655
avg min 74.49711067222589 52.58210811206203
avg avg 74.49711067222589 52.80886548167655
best e_cal: 89.51795772072269
best e: 86.2921512691098


In [63]:
#frank

ops = ["max","min","avg"]

best_e = 0
best_e_cal = 0

for o1 in ops:
    for o2 in ops:
        e, e_cal = getP(o1,o2)
        print(o1, o2, e_cal, e)
        if e > best_e:
            best_e = e
        if e_cal > best_e_cal:
            best_e_cal = e_cal

print("best e_cal:", best_e)
print("best e:", best_e_cal)

max max 80.06683124478164 80.50571625817318
max min 77.11165408234449 77.78629182374479
max avg 80.55677041352962 80.65779438819953
min max 50.0 50.0
min min 50.0 50.0
min avg 50.0 50.0
avg max 70.41967419316362 67.80651469094616
avg min 67.97404460958481 51.97186509413844
avg avg 70.45337897183256 60.58610170349192
best e_cal: 80.55677041352962
best e: 80.65779438819953


In [71]:
#halu_s

ops = ["max","min","avg"]

best_e = 0
best_e_cal = 0

for o1 in ops:
    for o2 in ops:
        e, e_cal = getP(o1,o2)
        print(o1, o2, e_cal, e)
        if e > best_e:
            best_e = e
        if e_cal > best_e_cal:
            best_e_cal = e_cal

print("best e_cal:", best_e)
print("best e:", best_e_cal)

max max 67.9 68.05
max min 72.9 70.55
max avg 73.3 72.7
min max 50.949999999999996 50.0
min min 50.0 50.0
min avg 50.24999999999999 50.0
avg max 69.30000000000001 57.35
avg min 68.69999999999999 50.3
avg avg 72.05 52.949999999999996
best e_cal: 73.3
best e: 72.7


In [13]:
#halu_d

ops = ["max","min","avg"]

best_e = 0
best_e_cal = 0

for o1 in ops:
    for o2 in ops:
        e, e_cal = getP(o1,o2)
        print(o1, o2, e_cal, e)
        if e > best_e:
            best_e = e
        if e_cal > best_e_cal:
            best_e_cal = e_cal

print("best e_cal:", best_e)
print("best e:", best_e_cal)

max max 60.3 52.55
max min 63.74999999999999 51.7
max avg 64.70000000000000 65.24
min max 60.3 50.4
min min 59.80000000000001 50.14999999999999
min avg 62.150000000000006 50.14999999999999
avg max 60.45 50.74999999999999
avg min 63.25000000000001 50.24999999999999
avg avg 62.650000000000006 50.3
best e: 64.70000000000000
best e_cal: 65.24


In [80]:
#halu_q

ops = ["max","min","avg"]

best_e = 0
best_e_cal = 0

for o1 in ops:
    for o2 in ops:
        e, e_cal = getP(o1,o2)
        print(o1, o2, e_cal, e)
        if e > best_e:
            best_e = e
        if e_cal > best_e_cal:
            best_e_cal = e_cal

print("best e_cal:", best_e)
print("best e:", best_e_cal)

max max 63.99999999999999 52.2
max min 66.05000000000001 51.5
max avg 69.65000000000001 69.3
min max 60.3 50.05
min min 61.050000000000004 50.0
min avg 60.35 50.0
avg max 63.45 50.349999999999994
avg min 65.5 50.0
avg avg 63.949999999999996 50.05
best e_cal: 69.65000000000001
best e: 69.3
